# Create a dog

In [25]:
import pandas as pd

In [26]:
dog_table = pd.DataFrame(columns=[
    'dog_id', 'minute',
    'happiness', 'food', 'fat', 'affection',
    'alive', 'can_action_be_taken', 'last_action_taken', 'minutes_since_last_action'
])

dog_table = dog_table.append({
    'dog_id': 1, 'minute': 0,
    'happiness': 0.50, 'food': 0.50, 'fat': 0, 'affection': 0.50,
    'alive': True, 'can_action_be_taken': True, 'last_action_taken': None, 'minutes_since_last_action': 0
    }, ignore_index=True)

In [27]:
WALKING_TIME = 15
EATING_TIME = 1
PLAYING_TIME = 4

food_consumption_rate = 1.0 / (30 * 3600)
affection_consumption_rate = 1.0 / (50 * 3600)
walking_fat_converge_rate = 0.2
walking_affection_converge_rate = 0.4
playing_fat_converge_rate = 0.1
playing_affection_converge_rate = 0.20
eating_food_increase = 0.6
eating_fat_increase = 0.25

In [28]:
def apply_decreasing_rate(value: float, rate: float) -> float:
    """
    Apply a decreasing rate to a value
    :param value: current value
    :param rate: per second
    :return: updated value
    """
    return value - (60 * rate)

In [29]:
def converge(value: float, target: float, ratio: float) -> float:
    diff: float = (target - value) * ratio
    return value + diff

# Change state after a minute

In [30]:
def update_walking(dog_table_row):
    update_fat = converge(dog_table_row.iloc[0]['fat'], 0.0, walking_fat_converge_rate)
    update_affection = converge(dog_table_row.iloc[0]['affection'], 1.0, walking_affection_converge_rate)
    return (update_fat, update_affection)

In [31]:
def update_feeding(dog_table_row):
    update_food = min(dog_table_row.iloc[0]['food'] + eating_food_increase, 1.0)
    update_fat = min(dog_table_row.iloc[0]['fat'] + eating_fat_increase, 1.0)
    return (update_food, update_fat)

In [32]:
def update_playing(dog_table_row):
    update_fat = converge(dog_table_row.iloc[0]['fat'], 0.0, playing_fat_converge_rate)
    update_affection = converge(dog_table_row.iloc[0]['affection'], 1.0, playing_affection_converge_rate)
    return (update_fat, update_affection)

In [33]:
def update_food(dog_table_row):
    return max(
        0.0,
        apply_decreasing_rate(dog_table_row.iloc[0]['food'], food_consumption_rate)
    )

In [34]:
def update_affection(dog_table_row):
    return max(
        0.0,
        apply_decreasing_rate(dog_table_row.iloc[0]['affection'], affection_consumption_rate)
    )

In [35]:
def update_happiness(dog_table_row) -> float:  # between 0 and 1
    return min(
        dog_table_row.iloc[0]['food'],
        1.0 - dog_table_row.iloc[0]['fat'],
        dog_table_row.iloc[0]['affection']
    )

In [36]:
def update_alive(dog_table_row):
    return dog_table_row.iloc[0]['alive'] & (dog_table_row.iloc[0]['happiness'] > 0.0)

In [59]:
def new_state_after_1_minute(dog_table, action_taken='NO ACTION'):
    
    dog_table_row = dog_table.tail(1)
    new_state_dog_id = dog_table_row.iloc[0]['dog_id']

    new_state_fat = dog_table_row.iloc[0]['fat']
    new_state_last_action_taken = dog_table_row.iloc[0]['last_action_taken']
    new_state_can_action_be_taken = dog_table_row.iloc[0]['can_action_be_taken']
    new_state_minutes_since_last_action = dog_table_row.iloc[0]['minutes_since_last_action']
    
    new_state_minute = dog_table_row.iloc[0]['minute'] + 1
    new_state_minutes_since_last_action = dog_table_row.iloc[0]['minutes_since_last_action'] + 1
    
    new_state_food = update_food(dog_table_row)
    new_state_affection = update_affection(dog_table_row)
    new_state_happiness = update_happiness(dog_table_row)
    new_state_alive = update_alive(dog_table_row)
    
    if ((action_taken != 'NO ACTION') & (dog_table_row.iloc[0]['can_action_be_taken'])):
        new_state_last_action_taken = action_taken
        new_state_can_action_be_taken = False
        new_state_minutes_since_last_action = 0
    
    if ((dog_table_row.iloc[0]['last_action_taken'] == 'WALKING') &
        (dog_table_row.iloc[0]['minutes_since_last_action'] == WALKING_TIME)):
        new_state_fat, new_state_affection = update_walking(dog_table_row)
        new_state_last_action_taken = 'NO ACTION'
        new_state_can_action_be_taken = True
        new_state_minutes_since_last_action = 0        

    if ((dog_table_row.iloc[0]['last_action_taken'] == 'FEEDING') &
        (dog_table_row.iloc[0]['minutes_since_last_action'] == EATING_TIME)):
        new_state_food, new_state_fat = update_feeding(dog_table_row)
        new_state_last_action_taken = 'NO ACTION'
        new_state_can_action_be_taken = True
        new_state_minutes_since_last_action = 0

    if ((dog_table_row.iloc[0]['last_action_taken'] == 'PLAYING') &
        (dog_table_row.iloc[0]['minutes_since_last_action'] == PLAYING_TIME)):
        new_state_fat, new_state_affection = update_playing(dog_table_row)
        new_state_last_action_taken = 'NO ACTION'
        new_state_can_action_be_taken = True
        new_state_minutes_since_last_action = 0


    dog_table = dog_table.append({
        'dog_id': new_state_dog_id,
        'minute': new_state_minute,
        'happiness': new_state_happiness,
        'food': new_state_food,
        'fat': new_state_fat,
        'affection': new_state_affection,
        'alive': new_state_alive,
        'can_action_be_taken': new_state_can_action_be_taken, 
        'last_action_taken': new_state_last_action_taken,
        'minutes_since_last_action': new_state_minutes_since_last_action
        }, ignore_index=True)
    
    return dog_table

In [60]:
dog_table = pd.DataFrame(columns=[
    'dog_id', 'minute',
    'happiness', 'food', 'fat', 'affection',
    'alive', 'can_action_be_taken', 'last_action_taken', 'minutes_since_last_action'
])

dog_table = dog_table.append({
    'dog_id': 1, 'minute': 0,
    'happiness': 0.50, 'food': 0.50, 'fat': 0, 'affection': 0.50,
    'alive': True, 'can_action_be_taken': True, 'last_action_taken': 'NO ACTION', 'minutes_since_last_action': 0
    }, ignore_index=True)

In [61]:
dog_table = new_state_after_1_minute(dog_table)
dog_table = new_state_after_1_minute(dog_table, action_taken = 'FEEDING')
dog_table = new_state_after_1_minute(dog_table)
dog_table = new_state_after_1_minute(dog_table)
dog_table = new_state_after_1_minute(dog_table, action_taken = 'PLAYING')
dog_table = new_state_after_1_minute(dog_table, action_taken = 'FEEDING')
dog_table = new_state_after_1_minute(dog_table)
dog_table = new_state_after_1_minute(dog_table)
dog_table = new_state_after_1_minute(dog_table)
dog_table

,dog_id,minute,happiness,food,fat,affection,alive,can_action_be_taken,last_action_taken,minutes_since_last_action
0,1,0,0.500000,0.500000,0,0.500000,True,True,NO ACTION,0
1,1,1,0.500000,0.499444,0,0.499667,True,True,NO ACTION,1
2,1,2,0.499444,0.498889,0,0.499333,True,False,FEEDING,0
3,1,3,0.498889,0.498333,0,0.499000,True,False,FEEDING,1
4,1,4,0.498333,1.000000,0.25,0.498667,True,True,NO ACTION,0
5,1,5,0.498667,0.999444,0.25,0.498333,True,False,PLAYING,0
6,1,6,0.498333,0.998889,0.25,0.498000,True,False,PLAYING,1
7,1,7,0.498000,0.998333,0.25,0.497667,True,False,PLAYING,2
8,1,8,0.497667,0.997778,0.25,0.497333,True,False,PLAYING,3
9,1,9,0.497333,0.997222,0.25,0.497000,True,False,PLAYING,4


In [62]:
def choose_an_action(dog_table, possible_actions=['NO ACTION', 'WALKING', 'FEEDING', 'PLAYING']):
    action_taken = random.choices(
        possible_actions,
        k = 1,
        weights=[80, 10, 1, 10]
    )
    return action_taken[0]

In [63]:
import random

cnt = 0
cnt_limit = 31*24*60

while ((dog_table.tail(1).iloc[0]['alive']) & (cnt < cnt_limit)):
    cnt += 1
    
    action_taken = choose_an_action(dog_table)
    print(action_taken)
    
    dog_table = new_state_after_1_minute(dog_table, action_taken)
    
dog_table

NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
WALKING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
PLAYING
PLAYING
NO ACTION
NO ACTION
WALKING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
PLAYING
NO ACTION
WALKING
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO A

NO ACTION
NO ACTION
NO ACTION
NO ACTION
PLAYING
WALKING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
PLAYING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
FEEDING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
FEEDING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
NO ACTION
WALKING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
PLAYING
WALKING
NO ACTION
PLAYING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
PLAYING
NO ACTION
WALKING
WALKING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
PLAYING
NO ACTION
NO ACTION
NO ACTION
PLAYING
NO ACTION
NO ACTION
NO ACTION
NO

PLAYING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
PLAYING
NO ACTION
WALKING
WALKING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
WALKING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
PLAYING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
PLAYING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
NO ACTION
FEEDING
NO ACTION
PLAYING
PLAYING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
NO ACTION
WALKING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACT

FEEDING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
PLAYING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
PLAYING
NO ACTION
NO ACTION
NO ACTION
WALKING
PLAYING
NO ACTION
PLAYING
WALKING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
NO ACTION
PLAYING
NO ACTION
NO ACTION
PLAYING
NO ACTION
WALKING
NO ACTION
PLAYING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
NO ACTION
FEEDING
FEEDING
NO ACTION
NO ACTION
NO ACTION
PLAYING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
FEEDING
PLAYING
NO ACTION
NO ACTION
PLAYING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
PLAYING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACT

NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
PLAYING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
FEEDING
PLAYING
NO ACTION
NO ACTION
PLAYING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
PLAYING
NO ACTION
WALKING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
WALKING
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
WALKING
NO ACTION
WALKING
PLAYING
PLAYING
NO ACTION
WALKING
NO ACTION
NO ACTION
PLAYING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
PLAYING
PLAYING
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION
NO ACTION


KeyboardInterrupt: 

In [ ]:
from collections import Counter
Counter(dog_table['last_action_taken'])

In [ ]:
plt.plot(dog_table['minute'], dog_table['happiness'])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = dog_table['happiness']
bins = np.linspace(0, 1, 1000)

plt.hist(x, bins, alpha=0.5, label='happiness')
plt.title('happiness')
plt.legend(loc='upper right')
plt.show()